In [ ]:
import numpy as np
import matplotlib
import pandas as pd
import yfinance as yf
import matplotlib.pyplot
from qiskit import QuantumCircuit
from qiskit.circuit.library import RZGate, RXGate
from qiskit.circuit.library import RZZGate
from qiskit.quantum_info import SparsePauliOp
from qiskit.circuit.library import QAOAAnsatz
from qiskit.circuit import ParameterVector
from qiskit_aer import AerSimulator
from scipy.optimize import minimize
from itertools import product
import matplotlib.pyplot as plt
from qiskit.quantum_info import Statevector

tickers = ['AAPL', 'MSFT', 'META', 'TSLA', 'NVDA', 'AMZN']
data = yf.download(tickers, start="2021-01-01", end="2025-07-19", auto_adjust=False)

adj_close = data.xs(key='Adj Close', level='Price', axis=1)
returns = np.log(adj_close / adj_close.shift(1)).dropna()

returnv = returns.mean()
cov = returns.cov().values

costs = adj_close.iloc[-1].values
biggest_cost = np.max(costs)
norm_costs = costs / biggest_cost
c_tot = sum(costs)

tickers = ['AAPL', 'MSFT', 'META', 'TSLA', 'NVDA', 'AMZN']
B = 4
q = 0.5
A = 20
n = len(tickers)
p = 4
lam = 1

qc = QuantumCircuit(n)
h = np.zeros(n)
J = np.zeros((n, n))


def cost_hamiltonian(returnv, cov, A, q, B):
    n = len(returnv)
    h = np.zeros(n)
    J = np.zeros((n, n))
    sum_cov = cov.sum(axis=1)

    for i in range(n):
        h[i] = (lam / 2) * (-(1 - q) * returnv[i] + A * (2 * B - n) - q * sum_cov[i])
        for j in range(i + 1, n):
            J[i, j] = (lam / 2) * (q * cov[i, j] + A)

    return (h, J)


h, J = cost_hamiltonian(returnv, cov, A, q, B)


def hamiltonian_operators(h, J):
    n = len(h)
    gates = []
    weights = []
    final_gates = []
    vars = []

    for i in range(n):
        gates = ["I"] * n
        gates[i] = "Z"
        final_gates.append("".join(gates))
        weights.append(h[i])

    for i in range(n):
        for j in range(i + 1, n):
            vars = ["I"] * n
            vars[i] = "Z"
            vars[j] = "Z"
            final_gates.append("".join(vars))
            weights.append(J[i, j])

    return SparsePauliOp(final_gates, weights)


cost_paulis = hamiltonian_operators(h, J)
print(cost_paulis)

from qiskit_aer.primitives import Estimator


def cost_hamil(qc, y):
    for i in range(n):
        qc.append(RZGate(2 * y * h[i]), [i])
    for i in range(n):
        for j in range(i + 1, n):
            qc.append(RZZGate(2 * y * J[i, j]), [i, j])


def mixer(qc, b):
    for i in range(n):
        qc.append(RXGate(2 * b), [i])


def state(params):
    y = params[:p]
    b = params[p:]
    qc = QuantumCircuit(n)
    qc.h(range(n))

    for gamma, beta in zip(y, b):
        cost_hamil(qc, gamma)
        mixer(qc, beta)

    return Statevector.from_instruction(qc)


graph = []


def qaoa_expectation_exact(params):
    exp = float(state(params).expectation_value(cost_paulis).real)
    graph.append(exp)
    return exp


np.random.seed(42)
init = np.hstack([np.random.rand(p) * np.pi, np.random.rand(p) * (np.pi / 2)])

optimize = minimize(
    qaoa_expectation_exact,
    init,
    method='COBYLA',
    options={'maxiter': 1000, 'disp': True}
)

print("optimized cost:", optimize.fun)
print("optimal y:", optimize.x[:p])
print("optimal b:", optimize.x[p:])

probs = state(optimize.x).probabilities_dict()
best_answer = max(probs, key=probs.get)
print("best answer:", best_answer, "cost:", optimize.fun)

[*********************100%***********************]  6 of 6 completed
/var/folders/l9/c7rm6fyd7fxfcjr78w4z1jf00000gp/T/ipykernel_27832/3976141650.py:52: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  h[i] = (lam / 2) * (-(1 - q) * returnv[i] + A * (2 * B - n) - q * sum_cov[i])


SparsePauliOp(['ZIIIII', 'IZIIII', 'IIZIII', 'IIIZII', 'IIIIZI', 'IIIIIZ', 'ZZIIII', 'ZIZIII', 'ZIIZII', 'ZIIIZI', 'ZIIIIZ', 'IZZIII', 'IZIZII', 'IZIIZI', 'IZIIIZ', 'IIZZII', 'IIZIZI', 'IIZIIZ', 'IIIZZI', 'IIIZIZ', 'IIIIZZ'],
              coeffs=[19.99946831+0.j, 19.99937536+0.j, 19.99914515+0.j, 19.99939667+0.j,
 19.99859786+0.j, 19.99904491+0.j, 10.00005748+0.j, 10.00006334+0.j,
 10.00004886+0.j, 10.00008067+0.j, 10.00008899+0.j, 10.00009638+0.j,
 10.00006284+0.j, 10.00010769+0.j, 10.00009882+0.j, 10.00006857+0.j,
 10.00011951+0.j, 10.00009555+0.j, 10.00008911+0.j, 10.00007011+0.j,
 10.00015671+0.j])
optimized cost:
   Return from subroutine COBYLA because the MAXFUN limit has been reached.
 -42.41841976763512
optimal y: [2.18970802 2.96462586 2.03671234 1.88950359]
optimal b: [0.27419871 1.34286026 0.11343678 1.38808588]

   NFVALS = 1000   F =-4.241842E+01    MAXCV = 0.000000E+00
   X = 2.189708E+00   2.964626E+00   2.036712E+00   1.889504E+00   2.741987E-01
       1.342860E+00   